In [405]:
import nltk
nltk.download('nps_chat')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from sklearn.metrics import accuracy_score
import torch
import pandas as pd
from datetime import datetime
import torchvision
from torch.utils.tensorboard import SummaryWriter
from transformers import RobertaTokenizerFast,RobertaTokenizer, RobertaForSequenceClassification,RobertaForMaskedLM, TextClassificationPipeline,TFRobertaModel
import numpy as np


[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [119]:
posts = nltk.corpus.nps_chat.xml_posts()
X = [post.text for post in posts]
y  = ['question' if (post.get('class') == 'whQuestion' or post.get('class') == 'ynQuestion') else "other" for post in posts]
translate_dict = {'question':1, 'other':0}

y = [translate_dict[entry] for entry in y]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

# NLTK Naive Bayes Classifier

++ easy to understand

-- ignores word order / context ("is this it" is the same as "this is it") confuses statements and questions

-- huge reliance on signal words

In [120]:

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features


featuresets = [(dialogue_act_features(post.text), target) for post, target in list(zip(posts,y))]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.8731060606060606


In [121]:
set([post.get('class') for post in posts])

{'Accept',
 'Bye',
 'Clarify',
 'Continuer',
 'Emotion',
 'Emphasis',
 'Greet',
 'Other',
 'Reject',
 'Statement',
 'System',
 'nAnswer',
 'whQuestion',
 'yAnswer',
 'ynQuestion'}

In [122]:
classifier.show_most_informative_features(50)

Most Informative Features
             contains(?) = True                1 : 0      =     59.5 : 1.0
           contains(wtf) = True                1 : 0      =     56.8 : 1.0
         contains(whats) = True                1 : 0      =     54.6 : 1.0
       contains(anybody) = True                1 : 0      =     44.8 : 1.0
           contains(asl) = True                1 : 0      =     44.8 : 1.0
            contains(tx) = True                1 : 0      =     32.9 : 1.0
         contains(which) = True                1 : 0      =     32.9 : 1.0
           contains(who) = True                1 : 0      =     31.1 : 1.0
          contains(part) = True                0 : 1      =     30.0 : 1.0
          contains(doin) = True                1 : 0      =     26.9 : 1.0
        contains(purple) = True                1 : 0      =     26.9 : 1.0
           contains(how) = True                1 : 0      =     25.0 : 1.0
        contains(anyone) = True                1 : 0      =     23.4 : 1.0

In [123]:
line = "Do you enjoy high performance programming?"
print(classifier.classify(dialogue_act_features(line)))


1


In [124]:
line = "What do you think about high performance programming?"
print(classifier.classify(dialogue_act_features(line)))

1


In [125]:
line = "i am testing this"
print(classifier.classify(dialogue_act_features(line)))

0


# Roberta


In [175]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt')
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.weight', 'cl

In [195]:
from datasets import load_dataset

dataset = load_dataset("dutch_social")


Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [196]:
encoded_dataset = dataset.map(lambda e: tokenizer(e['full_text'], truncation=True, padding='max_length'), batched=True)


Loading cached processed dataset at C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f\cache-67764fbfef6076b9.arrow


  0%|          | 0/55 [00:00<?, ?ba/s]

  0%|          | 0/55 [00:00<?, ?ba/s]

In [197]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [198]:
dataloader = torch.utils.data.DataLoader(encoded_dataset, batch_size=32)

In [199]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [142]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [143]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:


ValueError: too many values to unpack (expected 2)

# word prediction

In [283]:
tokenizer = RobertaTokenizer.from_pretrained('pdelobelle/robbert-v2-dutch-base')
model = RobertaForMaskedLM.from_pretrained('pdelobelle/robbert-v2-dutch-base', return_dict=True)

In [284]:
sequence = f"Mijn opleiding is erg {tokenizer.mask_token}."


In [98]:
input = tokenizer.encode(sequence, return_tensors="pt").to( 'cuda' if torch.cuda.is_available() else 'cpu' )
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

In [99]:
with torch.no_grad():
    token_logits = model(input).logits

In [100]:
logits = token_logits[0, mask_token_index, :].squeeze()
prob = logits.softmax(dim=0)
values, indeces = prob.topk(k=15, dim=0)

for index, token in enumerate(tokenizer.convert_ids_to_tokens(indeces)):
    print(f"{token:20} | id = {indeces[index]:4} | p = {values[index]}")

Ġgoed                | id =   76 | p = 0.26623669266700745
Ġleuk                | id =  407 | p = 0.11775296181440353
Ġgeslaagd            | id = 5008 | p = 0.07070890069007874
Ġuitgebreid          | id = 1607 | p = 0.046719152480363846
Ġprettig             | id = 2918 | p = 0.04093535244464874
Ġfijn                | id = 1158 | p = 0.03603675216436386
Ġintensief           | id = 6537 | p = 0.028142478317022324
Ġleerzaam            | id = 17557 | p = 0.02439124509692192
Ġgewaardeerd         | id = 8312 | p = 0.019797157496213913
Ġkort                | id =  989 | p = 0.019386067986488342
Ġinteressant         | id = 2349 | p = 0.01641271635890007
Ġwaardevol           | id = 10008 | p = 0.016243360936641693
Ġprofessioneel       | id = 3802 | p = 0.01359760481864214
Ġdivers              | id = 9014 | p = 0.013144192285835743
Ġmooi                | id =  384 | p = 0.011341879144310951


# test
https://github.com/iPieter/robbert

In [285]:
from datasets import load_dataset

dataset = load_dataset("dutch_social")

Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [286]:
MODEL_NAME = "pdelobelle/robbert-v2-dutch-base"
MAX_LEN = 512
ARTIFACTS_PATH = '../artifacts/'

BATCH_SIZE = 8 * strategy.num_replicas_in_sync
EPOCHS = 3

if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)

In [287]:
def roberta_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [288]:
X_train = roberta_encode(dataset['train']['full_text'], tokenizer)

In [289]:
X_test = roberta_encode(dataset['test']['full_text'], tokenizer)

In [290]:
y_train = dataset['train']['label']
y_test = dataset['test']['label']

In [291]:
y_train = np.asarray(y_train, dtype='int32')
y_test = np.asarray(y_test, dtype='int32')

In [292]:
import os

In [293]:
# Detect hardware, return appropriate distribution strategy (you can see that it is pretty easy to set up).
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set (always set in Kaggle)
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [294]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt')
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.weight', 'cl

In [295]:
def build_model(n_categories):
    with strategy.scope():
        input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
        input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
        input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

        # Import RoBERTa model from HuggingFace
        roberta_model = TFRobertaModel.from_pretrained(MODEL_NAME)
        x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

        # Huggingface transformers have multiple outputs, embeddings are the first one,
        # so let's slice out the first position
        x = x[0]

        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

        model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(lr=1e-5),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

        return model

In [296]:
with strategy.scope():
    model = build_model(n_categories)
    model.summary()

ResourceExhaustedError: Exception encountered when calling layer "roberta" (type TFRobertaMainLayer).

OOM when allocating tensor with shape[40000,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]

Call arguments received:
  • input_ids=tf.Tensor(shape=(3, 5), dtype=int32)
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=True
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False
  • kwargs=<class 'inspect._empty'>

In [ ]:
with strategy.scope():
    history = model.fit(X_train,y_train,
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=1,
                        validation_data=(X_test,y_test))

# test 2
https://jesusleal.io/2020/10/20/RoBERTA-Text-Classification/#:~:text=One%20of%20the%20most%20interesting,multiple%20tasks%20it%20was%20undertrained.

In [494]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [495]:
train_data, test_data = datasets.load_dataset('imdb', split =['train[:100]', 'test[:100]'])

Reusing dataset imdb (C:\Users\Emiel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [496]:
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base", model_max_length=512, return_tensors='pt', num_labels = 2)
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")

loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/vocab.json from cache at C:\Users\Emiel/.cache\huggingface\transformers\e338cb20bd645675d888a6469d1b7bfe472e69a5c93052006528c9ae4f856e44.43004df1e5c2251acdd15f077a74063dea7f0082e895ca8978bc6876d97c918b
loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/merges.txt from cache at C:\Users\Emiel/.cache\huggingface\transformers\0cc0d13f8f47c5f67868f6ea3638b66b788e833af7323164ba7d3e1c92f49f42.15e46d82fe3fef52038bdadabeb8cca392378e0966fa780352e52eaa8301f2ba
loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/pdelobelle/robbert-v2-dutch-base/resolve/main/special_tokens_map.json from cache at C:\Users\Emiel/.cache\huggingface\transformers\fc901152bccc6123aa5ca59de0c36b6e68e0796b4d7875a0e4e9744aab712cd2.a11ebb04664c067c8fe5ef8f8068b0f721263414a26058692f7b2e4ba2a1b342
loading file htt

In [497]:
train_data = train_data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
test_data = test_data.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [498]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [499]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [500]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = 'test',
    num_train_epochs=3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,    
    per_device_eval_batch_size= 8,
    evaluation_strategy = "steps",
    disable_tqdm = False,
    load_best_model_at_end=False,
    warmup_steps=0,
    weight_decay=0.01,
    fp16 = False,
    dataloader_num_workers = 8,
    run_name = 'roberta-classification')

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [501]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [502]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 3
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3, training_loss=0.3650932312011719, metrics={'train_runtime': 692.5797, 'train_samples_per_second': 0.433, 'train_steps_per_second': 0.004, 'total_flos': 69461318615040.0, 'train_loss': 0.3650932312011719, 'epoch': 2.64})